In [1]:
import tensorflow as tf
import numpy as np
import rocket_tools as rt
#from keras import backend 
#from keras.backend import learning_phase
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"  

Using TensorFlow backend.


In [2]:
def train_model(model, writer_path, sess, model_path, steps=60000, batch_size=32, image_size=128, n_images=3):    
    x_input = tf.placeholder(tf.float32, 
                             [None, image_size, image_size, n_images], 
                             name='input')
    y_true = tf.placeholder(tf.float32, 
                            [None, image_size, image_size, n_images], 
                            name='mask_true')
    y_pred = model(x_input) 
    
    loss_op = -rt.losses.dice_coef(y_true, y_pred)
    adam = tf.train.AdamOptimizer()
    train_op = adam.minimize(loss_op)
    
    with tf.variable_scope('metrics'):
        tf.summary.scalar('dice', rt.losses.dice_coef(y_true, y_pred))
        tf.summary.scalar('jacard', rt.losses.jacard_coef(y_true, y_pred)) 
        tf.summary.scalar('accuracy', rt.metrics.accuracy(y_true, y_pred))
        tf.summary.scalar('IoU', rt.metrics.IoU(y_true, y_pred))
        tf.summary.scalar('log_loss', loss_op)
        tf.summary.image('input', x_input[:1, :, :, :1])
        tf.summary.image('pred', y_pred[:1, :, :, :1])
        tf.summary.image('mask', y_true[:1, :, :, :1])
        summary_op = tf.summary.merge_all()
        writer = tf.summary.FileWriter(writer_path)
    
    
    
    image_generator = rt.image_gen.ImageGenerator(image_size=(image_size, image_size))
    batch_generator = rt.batch_generator.BatchGenerator(image_generator, batch_size, 
                                                        snr_mean=5, snr_std=2, n_images=n_images)
    
    # initialize training session
    sess = tf.InteractiveSession()
    sess.run(tf.global_variables_initializer())
   
    print('started training')
    for step in range(steps):
        x, y = next(batch_generator)
        _, summary = sess.run([train_op, summary_op], 
                              feed_dict={
                                  x_input: x, 
                                  y_true: y 
                              })
        writer.add_summary(summary, step) 
        
    print('finished training')
    saver = tf.train.Saver()
    saver.save(sess, model_path)


In [3]:
if __name__ == "__main__":
    # parsing
    #from argparse import ArgumentParser
    #parser = ArgumentParser()

    #parser.add_argument('--snr', type=float, required=True)
    #options = parser.parse_args()
    
    tf.reset_default_graph()
      
    image_size = 128
    n_images = 3

    TB_PATH = './../tensorboard/u_net/image_size={}_snr_all'.format(image_size)
    MODEL_PATH = './../saved_models/u_net_image_size={}_snr=_all'.format(image_size)
    os.system('rm -rf {}'.format(TB_PATH))
    
    model = rt.model.U_NET(input_=(image_size, image_size, n_images))
    sess = tf.InteractiveSession()

    train_model(model, writer_path=TB_PATH, model_path=MODEL_PATH, sess=sess)


started training


InvalidArgumentError: You must feed a value for placeholder tensor 'batch_normalization_1/keras_learning_phase' with dtype bool
	 [[Node: batch_normalization_1/keras_learning_phase = Placeholder[dtype=DT_BOOL, shape=<unknown>, _device="/job:localhost/replica:0/task:0/gpu:0"]()]]
	 [[Node: Neg/_43 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_2249_Neg", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op u'batch_normalization_1/keras_learning_phase', defined at:
  File "/usr/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/dist

In [5]:
model = rt.model.U_NET(input_=(128, 128, 3))
sess = tf.InteractiveSession()
saver = tf.train.Saver()
saver.restore(sess, './../saved_models/u_net_image_size=128_snr=_all')

INFO:tensorflow:Restoring parameters from ./../saved_models/u_net_image_size=128_snr=_all


In [10]:
x_input = tf.placeholder(tf.float32, [None, 128, 128, 3], 
                             name='input')
y_pred = model(x_input)
pred = y_pred.eval(feed_dict={x_input:-0.5*np.ones([1,128,128,3], dtype='float32'), learning_phase():1})

In [11]:
image_generator = rt.image_gen.ImageGenerator(image_size=(128, 128))
batch_generator = rt.batch_generator.BatchGenerator(image_generator, batch_size=64, snr_mean=5, snr_std=2, n_images=3)

x, y =next(batch_generator)
pred_s = sess.run(y_pred, feed_dict={x_input: x, learning_phase(): 1})